In [1]:
import pandas as pd
import requests
from datetime import datetime
from datetime import timedelta
import time
import re
from bs4 import BeautifulSoup
from datetime import datetime


In [2]:
def get_link(dir, start_date, end_date=False, Moscow_arpts=True, airport = '',):
    '''
    Функция возвращает список сслылок с сайта http://mvsm.ru/Tablo.aspx для последующего парсинга, принимая на вход 
    следующие аргументы:
    - 'dir' - направление рейса. 0 - вылет, 1 - прилет
    - start_date - начальная дата сбора информации (формат - YYYY-mm-dd), самое раннее значение - 2021-12-31.
    - end_date - дата завершения сбора информации (формат - YYYY-mm-dd). По умолчанию - текущие дата и время.
    - Moscow_arpts - по умолчанию - True, собирает информацию по аэропортам Домодедово, Шереметьево, Внуково. 
      При параметре False требуется указание параметра airports
    - airport - принимает на вход одно значение: 'DME'(Домодедово), 'SVO'(Шереметьево) или 'VKO'(Внуково).
    '''
    
    URL = 'http://mvsm.ru/Tablo.aspx'
    links_list = list()
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if end_date:
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    else:
        end_date = datetime.now()
        
    while start_date <= end_date:
        link = str(URL + (f'?Dir={dir}&Date={start_date.strftime("%Y-%m-%d")}&Hour={start_date.hour}'))
        if Moscow_arpts:
            link += (f'&City=Москва')
        if airport:
            link += (f'&Airport={airport}')
        links_list.append(link)
        
        start_date += timedelta(hours=6)
    
    return links_list 

# пример вывода URL-адреса с параметрами: http://mvsm.ru/Tablo.aspx?Dir=1&Date=2023-02-08&Hour=23&City=Москва&Airport=DME
    

In [3]:
res_links = get_link(0, '2022-01-01', '2023-03-01')

In [4]:
def get_flight_info_res(links_list, hide_joined_flights=True):
    
    check_time = None
    check_dep_airport = None
    check_arr_airport = None

    res_flights_info = pd.DataFrame()
    try:
        for link in links_list:
            try:
                response = requests.get(link)
                time.sleep(0.3)
                soup = BeautifulSoup(response.text, 'lxml')
                flights_info = soup.find_all('tr')           

                for el in list(flights_info)[30:]:
                    if el.find('a'):

                        fl_id = el.find('a', alt="Подписка").get('href')
                        fl_id = str(fl_id)[-8:]
                        if fl_id not in res_flights_info.values:                            

                            company_name = el.find('td', align='center')['title']
                            
                            company_code_pattern = str(re.search(r'\"Aircompany.aspx\?Code=\w+"', str(el)).group()).replace('"', '')
                            company_code = el.find('a', href=company_code_pattern).text
       
                            flight_number_pattern = str(re.search(r'\"FlightOnDate.aspx\?.*"', str(el)).group()).replace('"', '')
                            fl_string = str(flight_number_pattern[-26:-19])
                            flight_number = re.search(r'\s[0-9]{2,5}', fl_string).group().strip()

                            completed_flight = True
                            times_list = list()
                            time_pattern = '\d{2}.\d{2}.\d{4}\s\d{1,2}:\d{2}:\d{2}'
                            for i in el.find_all('td', align="center"):
                                if re.findall(time_pattern, i.text):
                                    times_list.append(str(re.search(time_pattern, i.text).group()))
                                if i.get_text(strip=True) == 'Отменен':
                                    completed_flight = False
                                    break
                                pass
                            plan_time, fact_time = times_list

                            aircraft = None
                            aircraft_pattern = '[АБВЕИЛМСABCDEFJLM]{1}\d\w{1,2}|32[01]{1}|Boeing\s\w+|Sukhoi\s\w+\s\w+|SSJ\d{2,3}'
                            for j in el.find_all('td', align="center"):
                                if re.search(aircraft_pattern, j.text):
                                    aircraft = (re.search(aircraft_pattern, j.text).group().replace('Б', 'B')
                                                .replace('В', 'B').replace('Е', 'E').replace('oeing', '').replace('320', 'A 320')
                                                .replace('AA', 'A').replace('А', 'A').replace('irbus', '')
                                                .replace('M83', 'MD83').replace('Sukhoi Superjet', 'SSJ')
                                                .replace(' ', '').replace('21N', '321').replace('20N', '320')
                                                .replace('B77W', 'B777').replace('J200', 'CRJ200').replace('E75L', 'E175')
                                                .replace('b38m', 'B738').replace('L76', 'Il76').replace('B39M', 'B739')
                                               )
                                    break
                                pass                                    
                                                      
                            airports_list = list()
                            for i in el.find_all('td', align="left"):
                                airports_list.append(str(i.get_text(strip=True)))       
                                
                            dep_airport = airports_list[0].upper()
                            arr_airport = airports_list[2].upper()
                    
                            if hide_joined_flights:
                                if (fact_time != check_time) and (arr_airport != check_arr_airport) and (dep_airport != check_dep_airport):            
                                    row = {'Flight_id': fl_id, 'Company_name': company_name, 'Company_code': company_code,
                                           'Flight_number': flight_number, 'Plan_time': plan_time, 'Fact_time': fact_time,
                                           'Completed_flight': completed_flight, 'Aircraft': aircraft, 
                                           'Departure_airport': dep_airport, 'Arrival_airport': arr_airport}

                                    check_time = fact_time
                                    check_arr_airport = arr_airport
                                    check_dep_airport = dep_airport
                                    
                                    res_flights_info = pd.concat([res_flights_info, pd.DataFrame([row])])
                                    continue

                            else:
                                row = {'Flight_id': fl_id, 'Company_name': company_name, 'Company_code': company_code, 'Flight_number': flight_number,
                                       'Plan_time': plan_time, 'Fact_time': fact_time, 'Completed_flight': completed_flight, 'Aircraft': aircraft,
                                       'Departure_airport': dep_airport, 'Arrival_airport': arr_airport}
                                
                                res_flights_info = pd.concat([res_flights_info, pd.DataFrame([row])])
                                pass

            except Exception as e:
                print(link[37:55], end = ', ')
                pass
                            
    finally:
        return res_flights_info

In [5]:
result = get_flight_info_res(res_links, False)

C:\Users\Kirill\AppData\Local\Temp\ipykernel_6020\249338194.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if fl_id not in res_flights_info.values:


2022-01-01&Hour=0&, 2022-01-01&Hour=6&, 2022-01-01&Hour=12, 2022-01-01&Hour=18, 2022-01-02&Hour=0&, 2022-01-02&Hour=6&, 2022-01-02&Hour=12, 2022-01-02&Hour=18, 2022-01-03&Hour=0&, 2022-01-03&Hour=6&, 2022-01-03&Hour=12, 2022-01-03&Hour=18, 2022-01-04&Hour=0&, 2022-01-04&Hour=6&, 2022-01-04&Hour=12, 2022-01-04&Hour=18, 2022-01-05&Hour=0&, 2022-01-05&Hour=6&, 2022-01-05&Hour=12, 2022-01-05&Hour=18, 2022-01-06&Hour=0&, 2022-01-06&Hour=6&, 2022-01-06&Hour=12, 2022-01-06&Hour=18, 2022-01-07&Hour=0&, 2022-01-07&Hour=6&, 2022-01-07&Hour=12, 2022-01-07&Hour=18, 2022-01-08&Hour=0&, 2022-01-08&Hour=12, 2022-01-08&Hour=18, 2022-01-09&Hour=0&, 2022-01-09&Hour=6&, 2022-01-09&Hour=12, 2022-01-09&Hour=18, 2022-01-10&Hour=6&, 2022-01-10&Hour=12, 2022-01-10&Hour=18, 2022-01-11&Hour=0&, 2022-01-11&Hour=6&, 2022-01-11&Hour=12, 2022-01-11&Hour=18, 2022-01-12&Hour=0&, 2022-01-12&Hour=6&, 2022-01-12&Hour=18, 2022-01-13&Hour=0&, 2022-01-13&Hour=6&, 2022-01-13&Hour=12, 2022-01-13&Hour=18, 2022-01-14&Hour=0&, 

2022-04-18&Hour=12, 2022-04-18&Hour=18, 2022-04-19&Hour=0&, 2022-04-19&Hour=6&, 2022-04-19&Hour=12, 2022-04-19&Hour=18, 2022-04-20&Hour=0&, 2022-04-20&Hour=6&, 2022-04-20&Hour=12, 2022-04-20&Hour=18, 2022-04-21&Hour=0&, 2022-04-21&Hour=6&, 2022-04-21&Hour=12, 2022-04-21&Hour=18, 2022-04-22&Hour=0&, 2022-04-22&Hour=6&, 2022-04-22&Hour=12, 2022-04-22&Hour=18, 2022-04-23&Hour=0&, 2022-04-23&Hour=6&, 2022-04-23&Hour=12, 2022-04-23&Hour=18, 2022-04-24&Hour=0&, 2022-04-24&Hour=6&, 2022-04-24&Hour=12, 2022-04-24&Hour=18, 2022-04-25&Hour=0&, 2022-04-25&Hour=6&, 2022-04-25&Hour=12, 2022-04-25&Hour=18, 2022-04-26&Hour=0&, 2022-04-26&Hour=6&, 2022-04-26&Hour=12, 2022-04-26&Hour=18, 2022-04-27&Hour=0&, 2022-04-27&Hour=6&, 2022-04-27&Hour=12, 2022-04-28&Hour=0&, 2022-04-28&Hour=6&, 2022-04-28&Hour=12, 2022-04-29&Hour=0&, 2022-04-29&Hour=6&, 2022-04-29&Hour=12, 2022-04-29&Hour=18, 2022-04-30&Hour=0&, 2022-04-30&Hour=6&, 2022-04-30&Hour=12, 2022-04-30&Hour=18, 2022-05-01&Hour=0&, 2022-05-01&Hour=6&, 

2022-07-31&Hour=0&, 2022-07-31&Hour=6&, 2022-07-31&Hour=12, 2022-07-31&Hour=18, 2022-08-01&Hour=0&, 2022-08-01&Hour=6&, 2022-08-01&Hour=12, 2022-08-01&Hour=18, 2022-08-02&Hour=0&, 2022-08-02&Hour=6&, 2022-08-02&Hour=12, 2022-08-02&Hour=18, 2022-08-03&Hour=0&, 2022-08-03&Hour=6&, 2022-08-03&Hour=12, 2022-08-03&Hour=18, 2022-08-04&Hour=0&, 2022-08-04&Hour=6&, 2022-08-04&Hour=12, 2022-08-04&Hour=18, 2022-08-05&Hour=0&, 2022-08-05&Hour=6&, 2022-08-05&Hour=12, 2022-08-05&Hour=18, 2022-08-06&Hour=0&, 2022-08-06&Hour=6&, 2022-08-06&Hour=12, 2022-08-06&Hour=18, 2022-08-07&Hour=0&, 2022-08-07&Hour=6&, 2022-08-07&Hour=12, 2022-08-07&Hour=18, 2022-08-08&Hour=0&, 2022-08-08&Hour=6&, 2022-08-08&Hour=12, 2022-08-08&Hour=18, 2022-08-09&Hour=0&, 2022-08-09&Hour=6&, 2022-08-09&Hour=12, 2022-08-09&Hour=18, 2022-08-10&Hour=0&, 2022-08-10&Hour=6&, 2022-08-10&Hour=12, 2022-08-10&Hour=18, 2022-08-11&Hour=0&, 2022-08-11&Hour=6&, 2022-08-11&Hour=12, 2022-08-11&Hour=18, 2022-08-12&Hour=12, 2022-08-12&Hour=18, 

2022-11-12&Hour=18, 2022-11-13&Hour=0&, 2022-11-13&Hour=6&, 2022-11-13&Hour=18, 2022-11-14&Hour=0&, 2022-11-14&Hour=6&, 2022-11-14&Hour=12, 2022-11-14&Hour=18, 2022-11-15&Hour=0&, 2022-11-15&Hour=6&, 2022-11-15&Hour=12, 2022-11-15&Hour=18, 2022-11-16&Hour=0&, 2022-11-16&Hour=6&, 2022-11-16&Hour=12, 2022-11-16&Hour=18, 2022-11-17&Hour=0&, 2022-11-17&Hour=6&, 2022-11-17&Hour=12, 2022-11-17&Hour=18, 2022-11-18&Hour=0&, 2022-11-18&Hour=6&, 2022-11-18&Hour=12, 2022-11-18&Hour=18, 2022-11-19&Hour=0&, 2022-11-19&Hour=6&, 2022-11-19&Hour=18, 2022-11-20&Hour=0&, 2022-11-20&Hour=6&, 2022-11-20&Hour=12, 2022-11-20&Hour=18, 2022-11-21&Hour=0&, 2022-11-21&Hour=6&, 2022-11-21&Hour=12, 2022-11-21&Hour=18, 2022-11-22&Hour=0&, 2022-11-22&Hour=6&, 2022-11-22&Hour=12, 2022-11-22&Hour=18, 2022-11-23&Hour=0&, 2022-11-23&Hour=6&, 2022-11-23&Hour=12, 2022-11-23&Hour=18, 2022-11-24&Hour=0&, 2022-11-24&Hour=6&, 2022-11-24&Hour=12, 2022-11-24&Hour=18, 2022-11-25&Hour=0&, 2022-11-25&Hour=6&, 2022-11-25&Hour=12, 

2023-02-26&Hour=0&, 2023-02-26&Hour=6&, 2023-02-26&Hour=12, 2023-02-26&Hour=18, 2023-02-27&Hour=0&, 2023-02-27&Hour=6&, 2023-02-27&Hour=12, 2023-02-27&Hour=18, 2023-02-28&Hour=0&, 2023-02-28&Hour=6&, 2023-02-28&Hour=12, 2023-02-28&Hour=18, 2023-03-01&Hour=0&, 

In [6]:
result = pd.DataFrame(result)

In [7]:
rus_companies_list = ['Аэрофлот', 'Россия', 'Победа', 'S7', 'Уральские', 'Ural Airlines', 'Северный', 'Nordwind',
                      'Ямал', 'Pegas Fly', 'IrAero', 'Ираэро', 'Ред Вингс', 'Red Wings', 'Utair', 'Azur Air', 'Royal Flight', 
                      'Smartavia', 'Нордавиа', 'NordStar', 'Северсталь', 'Руслайн', 'Алроса', 'Ижавиа', 'Азимут', 'Якутия', 
                      'SmartWings', 'Газпром', 'ЮВТ', 'ВОЛОГОДСКОЕ', 'Северо-Запад', 'I Fly', 'IFly', 'РусДжет', 'Таймыр'
                     ]

def search_rus_aircompanies(company_name):
    for i in rus_companies_list:
        if i in company_name:
            return 1    

In [8]:
result['rus_company'] = result['Company_name'].apply(search_rus_aircompanies)
result.tail()

,Flight_id,Company_name,Company_code,Flight_number,Plan_time,Fact_time,Completed_flight,Aircraft,Departure_airport,Arrival_airport,rus_company
0,19383657,Руслайн,РГ,4247,01.03.2023 0:20:00,01.03.2023 0:20:00,True,None,МОСКВА (ВНУКОВО),СУРГУТ (СУРГУТ),1.0
0,19383658,Utair,UT,247,01.03.2023 0:20:00,01.03.2023 0:20:00,True,B737,МОСКВА (ВНУКОВО),СУРГУТ (СУРГУТ),1.0
0,19383659,Utair,UT,453,01.03.2023 0:25:00,01.03.2023 0:25:00,True,None,МОСКВА (ВНУКОВО),ТЮМЕНЬ (РОЩИНО),1.0
0,19383660,Руслайн,РГ,4453,01.03.2023 0:25:00,01.03.2023 0:25:00,True,None,МОСКВА (ВНУКОВО),ТЮМЕНЬ (РОЩИНО),1.0
0,19373978,Победа,DP,6501,01.03.2023 0:30:00,01.03.2023 0:40:28,True,B737,МОСКВА (ШЕРЕМЕТЬЕВО),УФА (УФА),1.0


In [9]:
result = result.sort_values(['Plan_time', 'Fact_time', 'Departure_airport', 'Arrival_airport', 'Aircraft', 'rus_company'])

In [10]:
result.shape

(230539, 11)

In [11]:
result.to_csv('MOW_DEPARTURES_20220101_20230301.csv', sep='\t', encoding='utf-8')

In [12]:
result.to_excel('MOW_DEPARTURES_20220101_20230301.xlsx', encoding='utf-8')